In [2]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from pathlib import Path
import scipy.sparse as sparse
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.data import Dataset

In [3]:
df_sub = pd.read_csv(Path.cwd() / 'data' / 'sample_submission.csv')

In [4]:
df_train = pd.read_csv(Path.cwd() / 'data' / 'train.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df_test = pd.read_csv(Path.cwd() / 'data' / 'test.csv')

/home/alex/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
df_train.drop_duplicates(inplace=True)

In [56]:
df_train.userid.max()

127495

In [57]:
df_test.userid.max()

127495

In [7]:
train_data, test_data = train_test_split(df_train, random_state=32, shuffle=True)

In [22]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 828935 entries, 0 to 857894
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         828935 non-null  float64
 1   verified        828935 non-null  bool   
 2   reviewTime      828935 non-null  object 
 3   asin            828935 non-null  object 
 4   reviewerName    828771 non-null  object 
 5   reviewText      828661 non-null  object 
 6   summary         828769 non-null  object 
 7   unixReviewTime  828935 non-null  int64  
 8   vote            115991 non-null  object 
 9   style           429265 non-null  object 
 10  image           7026 non-null    object 
 11  userid          828935 non-null  int64  
 12  itemid          828935 non-null  int64  
 13  rating          828935 non-null  float64
dtypes: bool(1), float64(2), int64(3), object(8)
memory usage: 89.3+ MB


In [60]:
train_data.values[0]

array([5.0, True, '08 9, 2016', 'B000UUWECC', 'Dorothy Gomes',
       'Great\nproduct and service.', 'Great product and service',
       1470700800, nan, nan, nan, 3215, 4724, 1.0], dtype=object)

In [61]:
a = set(df_train.userid.values)

In [62]:
len(a)

127448

In [63]:
b = set(df_test.userid.values)

In [27]:
len(b)

109357

In [32]:
ratings_coo = sparse.coo_matrix((df_train['rating'].astype(int),
                                 (df_train['userid'],
                                  df_train['itemid'])))

In [33]:
ratings_coo.shape

(127496, 41320)

In [18]:
len(df_train['overall'].astype(int))

828935

In [21]:
len(df_train)

828935

In [34]:
user_feature_coo = sparse.coo_matrix((df_train['overall'].astype(int),
                                     (df_train['userid'],
                                     [0]*len(df_train))))

In [35]:
user_feature_coo.shape

(127496, 1)

In [36]:
ratings_coo.shape

(127496, 41320)

In [37]:
NUM_THREADS = 8 #число потоков
NUM_COMPONENTS = 60 #число параметров вектора 
NUM_EPOCHS = 25 #число эпох обучения

model = LightFM(learning_rate=0.1, loss='logistic',
                no_components=NUM_COMPONENTS)
model = model.fit(ratings_coo, user_features=user_feature_coo, epochs=NUM_EPOCHS, 
                  num_threads=NUM_THREADS)

In [45]:
preds = model.predict(test_data.userid.values,
                      test_data.itemid.values, user_features=user_feature_coo)

In [46]:
roc_auc_score(test_data.rating,preds)

0.735878893591469

In [38]:
preds_sub = model.predict(df_test.userid.values,
                      df_test.itemid.values, user_features=user_feature_coo)

In [39]:
len(preds_sub)

285965

In [40]:
preds_sub.min(), preds_sub.max()

(-490.41937255859375, 885.7655029296875)

In [41]:
normalized_preds = (preds_sub - preds_sub.min())/(preds_sub - preds_sub.min()).max()

In [42]:
normalized_preds.min(), normalized_preds.max()

(0.0, 1.0)

In [43]:
df_sub['rating']= normalized_preds

In [44]:
df_sub.to_csv('submission_log.csv', index=False)